In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from pydrake.geometry.optimization import HPolyhedron
from pydrake.trajectories import PiecewisePolynomial

from spp.bezier import BezierSPP
from spp.linear import LinearSPP

In [ ]:
from scipy.spatial import ConvexHull

from models.env_2d import obstacles, vertices

def make_hpolytope(V):
    ch = ConvexHull(V)
    return HPolyhedron(ch.equations[:, :-1], - ch.equations[:, -1])

regions = [make_hpolytope(V) for V in vertices]

x_start = np.array([0, 0])
x_goal = np.array([4.8, 4.2])

In [ ]:
x_min = np.min(np.vstack(vertices), axis=0)
x_max = np.max(np.vstack(vertices), axis=0)

def environment_plot_setup():
    
    plt.figure(figsize=(2.5,4))
    plt.axis('square')
    
    plt.xlim([x_min[0], x_max[0]])
    plt.ylim([x_min[1], x_max[1]])
    
    tick_gap = .2
    n_ticks = lambda x_min, x_max: int((x_max - x_min) / tick_gap) + 1
    x_ticks = np.linspace(x_min[0], x_max[0], n_ticks(x_min[0], x_max[0]))
    y_ticks = np.linspace(x_min[1], x_max[1], n_ticks(x_min[1], x_max[1]))
    plt.xticks(x_ticks)
    plt.yticks(y_ticks)
    
    label_gap = .5
    keep_label = lambda t: np.isclose(t % label_gap, 0) or np.isclose(t % label_gap, label_gap)
    x_labels = [int(t) if keep_label(t) else '' for t in x_ticks]
    y_labels = [int(t) if keep_label(t) else '' for t in y_ticks]
    plt.gca().set_xticklabels(x_labels)
    plt.gca().set_yticklabels(y_labels)
    
    plt.grid()

In [ ]:
environment_plot_setup()

for O in obstacles:
    plt.fill(*O.T, fc='lightcoral', ec='k', zorder=4)

plt.plot(*x_start, 'kx')
plt.plot(*x_goal, 'kx')

plt.text(.1, -.4, '$q_0$', ha='center')
plt.text(4.7, 3.8, '$q_T$', ha='center')

plt.savefig('setup.pdf', bbox_inches='tight')

In [ ]:
environment_plot_setup()

for V in vertices:
    plt.fill(*V.T, fc='lightcyan', ec='k', zorder=4)

plt.savefig('decomposition.pdf', bbox_inches='tight')

In [ ]:
n_samples = 500
relaxation = False
qdot_min = -1
qdot_max = 1

def solve_linear():
    spp = LinearSPP(regions)
    waypoints = spp.SolvePath(x_start, x_goal, relaxation)[0]
    t_knots = np.linspace(0, 1,  waypoints.shape[1])
    traj = PiecewisePolynomial.FirstOrderHold(t_knots, waypoints)
    times = np.linspace(
        traj.start_time(),
        traj.end_time(),
        n_samples
    )
    return traj, times

def solve_bezier(order, continuity, regularizer=None, hdot_min=1e-6):
    spp = BezierSPP(
        regions,
        order=order,
        continuity=continuity,
        hdot_min=hdot_min
    )
    spp.addTimeCost(1)
    spp.addVelocityLimits([qdot_min] * 2, [qdot_max] * 2)
    if regularizer is not None:
        spp.addAccelerationRegularization(*regularizer)
    traj = spp.SolvePath(x_start, x_goal, relaxation)[0]
    times = np.linspace(
        traj.start_time(),
        traj.end_time(),
        n_samples
    )
    return traj, times

In [ ]:
def plot_trajectory(traj, times):

    plt.figure(figsize=(2.5,4))

    for O in obstacles:
        plt.fill(*O.T, fc='lightcoral', ec='k', zorder=4)

    plt.plot(*x_start, 'kx')
    plt.plot(*x_goal, 'kx')

    plt.axis('square')
    plt.xlim([x_min[0], x_max[0]])
    plt.ylim([x_min[1], x_max[1]])
    
    plt.xticks(range(6))

    values = np.squeeze([traj.value(t) for t in times])
    plt.plot(*values.T, 'b', zorder=5)

    plt.grid(1)
    
plot_trajectory(*solve_linear())
plt.savefig('linear.pdf', bbox_inches='tight')

In [ ]:
def plot_velocity(traj, times, tol=np.inf, ylabel=None):

    plt.figure(figsize=(2.5,1.5))
    
    velocity = np.array([traj.EvalDerivative(t) for t in times])
    
#     plt.plot(times, np.ones(len(times)) * qdot_min, 'r--', linewidth=3)
#     plt.plot(times, np.ones(len(times)) * qdot_max, 'r--', linewidth=3)

    def plot_with_jumps(velocity, color):
        for i in range(len(times) - 3):
            dv = velocity[i + 2] - velocity[i + 1]
            style = '-' if abs(dv) < tol else ':'
            plt.plot(times[i+1:i+3], velocity[i+1:i+3], color=color, linestyle=style)
            
    plot_with_jumps(velocity[:,0], 'tab:blue')
    plot_with_jumps(velocity[:,1], 'tab:orange')
    
#     plt.plot(times[1:-1], velocity[1:-1, 0])
#     plt.plot(times[1:-1], velocity[1:-1, 1])

    plt.xlabel('Time $t$')
    plt.ylabel(ylabel)
    
    plt.grid()
    plt.xticks(np.arange(int(np.ceil(times[-1] / 2))) * 2)
    plt.yticks(np.linspace(qdot_min, qdot_max, 5))
    
    plt.xlim([times[0], times[-1]])

In [ ]:
traj, times = solve_bezier(1, 0)

plot_trajectory(traj, times)
plt.savefig('bezier_10.pdf', bbox_inches='tight')

plot_velocity(traj, times, tol=1e-1, ylabel='Velocity $\dot{q}$')
plt.savefig('bezier_10_vel.pdf', bbox_inches='tight')

In [ ]:
traj, times = solve_bezier(5, 2)

plot_trajectory(traj, times)
plt.savefig('bezier_52.pdf', bbox_inches='tight')

plot_velocity(traj, times)
plt.savefig('bezier_52_vel.pdf', bbox_inches='tight')

In [ ]:
regularizer = [1e-3, 1e-3]
hdot_min = 1e-3
traj, times = solve_bezier(5, 2, regularizer, hdot_min)

plot_trajectory(traj, times)
plt.savefig('bezier_52_reg.pdf', bbox_inches='tight')

plot_velocity(traj, times)
plt.savefig('bezier_52_reg_vel.pdf', bbox_inches='tight')